In [39]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [8]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

In [17]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [18]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [23]:
tf_fuji['VELOCITY']=np.nan
tf_fuji['VELOCITY_REL']=np.nan
tf_iron['VELOCITY']=np.nan
tf_iron['VELOCITY_REL']=np.nan

In [47]:
# tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
# tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]
# df = pd.DataFrame(tf_iron)
# print(df.max())

# Filter the data based on the condition
filtered_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]

filtered_tf_fuji = filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE'] == 1]
filtered_tf_iron = filtered_tf_iron[filtered_tf_iron['ROT_CURVE'] == 1]

# Find the maximum value of a specific column in the filtered data
max_value_fuji = filtered_tf_fuji['Z'].max()
max_value_iron = filtered_tf_iron['Z'].max()

print("Maximum value in tf_fuji:", max_value_fuji)
print("Maximum value in tf_iron:", max_value_iron)

Maximum value in tf_fuji: 0.173630148670864
Maximum value in tf_iron: 1.62543750380985


In [63]:
tf_fuji_rot = np.unique(tf_fuji[tf_fuji['ROT_CURVE']==1]['SGA_ID'])
tf_iron_rot = np.unique(tf_iron[tf_iron['ROT_CURVE']==1]['SGA_ID'])
print(tf_fuji_rot, tf_iron_rot)

 SGA_ID
-------
    123
    896
   1548
   1583
   1980
   2497
   5732
   5958
   5964
   6443
    ...
1421164
1421334
1424489
1427956
1429142
1429680
1430013
1431285
1434487
5001339
5002047
Length = 1267 rows  SGA_ID
-------
    123
    896
   1548
   1583
   1980
   2497
   5732
   5958
   5964
   6443
    ...
1421334
1424489
1427956
1429142
1429680
1430013
1431285
1434487
2000055
5001339
5002047
Length = 1298 rows


In [95]:
# find all unique target ID's for one galaxy
targets_fuji=tf_fuji[tf_fuji['SGA_ID']==  123]
targets_iron=tf_iron[tf_iron['SGA_ID']== 123]
targets=(vstack([targets_fuji,targets_iron]))
target_list=np.unique(targets['TARGETID'])
print(target_list)

     TARGETID    
-----------------
 1071233026228227
 1071233026228228
 1071233026228231
 1084427165761544
39633304835591852


In [55]:
centers_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
centers_tf_fuji

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2305843015479989956,173.45541945964,53.1254763873733,5976,sv1,0.00333404373962337,2.24812533738904e-07,0,1486700.56152642,fuji/healpix/sv1/other/59/5976/redrock-sv1-other-5976.fits,EXT,559707,173.4553483637811,53.12541571341898,1.0,6.3611093629270335e-15,1.3719474372226997e-13,0.0,1.0,nan,nan
39633062551620033,153.178857933669,39.3739445564769,5703,sv1,0.022711924294578,5.75568615760296e-07,0,641871.757860661,fuji/healpix/sv1/bright/57/5703/redrock-sv1-bright-5703.fits,EXT,1037949,153.17885793366855,39.3739445564769,1.0,0.0,0.0,0.0,0.0,nan,nan
39633062056692288,115.163713024568,39.2330431988692,6560,sv1,0.0112289665628835,2.5580485409679e-06,0,199314.054352144,fuji/healpix/sv1/bright/65/6560/redrock-sv1-bright-6560.fits,EXT,17643,115.16371302456768,39.2330431988692,1.0,1.1007441864020942e-14,2.0732856604950621e-13,0.0,0.0,nan,nan
39627865758633464,150.11281896177,3.37442274219938,27345,sv3,0.00687190356978907,1.5141461957797e-06,0,24405.1790848412,fuji/healpix/sv3/bright/273/27345/redrock-sv3-bright-27345.fits,EXT,1225398,150.1128189617695,3.3744227421993793,1.0,0.0,0.0,1.0,0.0,nan,nan
39627770296276005,219.956999318774,-0.718640392418429,25933,sv3,0.00585538786570846,4.19501768104605e-06,0,2858.02824933385,fuji/healpix/sv3/bright/259/25933/redrock-sv3-bright-25933.fits,EXT,166290,219.95699931877425,-0.7186403924184287,1.0,0.0,0.0,1.0,0.0,nan,nan
39633365250342941,178.583332219869,58.1831490603587,6007,sv2,0.053196839390209,6.46704883796473e-06,0,5668.41995278001,fuji/healpix/sv2/bright/60/6007/redrock-sv2-bright-6007.fits,TFT,1091594,178.58333221986857,58.18314906035869,1.0,0.0,0.0,0.0,0.0,nan,nan
39633404920070175,186.78389561027,61.618475942601,11250,sv3,0.0705627702950349,6.8084684234377e-06,0,2307.48744933307,fuji/healpix/sv3/bright/112/11250/redrock-sv3-bright-11250.fits,TFT,820765,186.78389561027012,61.61847594260104,1.0,0.0,0.0,0.0,0.0,nan,nan
39632991730794532,252.858128421553,35.5753130130782,9429,sv3,0.0641584566287161,7.48677701522306e-06,0,898.090423569083,fuji/healpix/sv3/bright/94/9429/redrock-sv3-bright-9429.fits,TFT,651270,252.85812842155318,35.57531301307821,1.0,0.0,0.0,1.0,0.0,nan,nan
39633220098064420,162.038766796335,48.3314519602365,5914,sv1,0.041278770975308,2.28758632187292e-06,0,129189.216787577,fuji/healpix/sv1/bright/59/5914/redrock-sv1-bright-5914.fits,TFT,1061166,162.0387667963351,48.3314519602365,1.0,3.1805546814635168e-15,4.2064617259532895e-13,0.0,0.0,nan,nan


In [57]:
centers_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]
centers_tf_iron

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY_REL,VELOCITY
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39632981425390961,222.111381963082,34.9980057574604,9187,main,0.0294233178741356,1.01512926413074e-05,0,21237.1530243214,iron/healpix/main/bright/91/9187/redrock-main-bright-9187.fits,EXT,492092,222.11138196308235,34.998005757460405,0.0,0.0,0.0,1.0,0.0,nan,nan
39633212137276827,175.319255631279,47.6908027474544,5593,main,0.010359363587595,1.28399688016051e-06,0,165308.385529607,iron/healpix/main/bright/55/5593/redrock-main-bright-5593.fits,EXT,519430,175.31925563127928,47.69080274745436,0.0,0.0,0.0,0.0,0.0,nan,nan
39632947015320877,269.023824643101,33.2084140231297,32713,main,0.0159236496100827,5.29297961077083e-06,0,109260.073396253,iron/healpix/main/bright/327/32713/redrock-main-bright-32713.fits,EXT,80048,269.0238246431006,33.20841402312973,0.0,0.0,0.0,0.0,0.0,nan,nan
2852223826526217,231.672830190053,40.5645147760559,9869,main,0.00874226946408794,3.93907247375542e-06,0,18038.6796703626,iron/healpix/main/dark/98/9869/redrock-main-dark-9869.fits,EXT,1371198,231.67283019005313,40.56451477605587,0.0,0.0,0.0,0.0,0.0,nan,nan
39633010814878547,216.086231258379,36.4613289316431,10524,main,0.027967980236947,7.68741137806728e-06,0,25608.7173414612,iron/healpix/main/bright/105/10524/redrock-main-bright-10524.fits,EXT,179329,216.08623125837948,36.461328931643074,0.0,0.0,0.0,0.0,0.0,nan,nan
39633429452556827,106.645087200257,63.8489485519007,7815,main,0.0142824781609945,3.07212636251081e-06,0,115559.587142028,iron/healpix/main/bright/78/7815/redrock-main-bright-7815.fits,EXT,599080,106.64508720025721,63.848948551900726,0.0,1.4180392320368964e-14,5.256999774323051e-13,0.0,0.0,nan,nan
39633062056692288,115.163713024568,39.2330431988692,6560,sv1,0.0112290864530619,2.55248321965483e-06,0,210806.64164253,iron/healpix/sv1/bright/65/6560/redrock-sv1-bright-6560.fits,EXT,17643,115.16371302456768,39.2330431988692,1.0,1.1007441864020942e-14,2.0732856604950621e-13,0.0,0.0,nan,nan
39632930946943252,210.650306403024,32.5103349112541,10357,main,0.0142975626621876,2.47037281141051e-06,0,77415.0852880366,iron/healpix/main/bright/103/10357/redrock-main-bright-10357.fits,EXT,747892,210.650306403024,32.51033491125415,0.0,2.3967871667907746e-14,8.569450059996153e-13,0.0,0.0,nan,nan
2852067974578181,210.650306403024,32.5103349112541,10357,main,0.0142602971000219,1.6354441573242e-06,0,128448.740757972,iron/healpix/main/dark/103/10357/redrock-main-dark-10357.fits,EXT,747892,210.650306403024,32.51033491125415,0.0,2.3967871667907746e-14,8.569450059996153e-13,0.0,0.0,nan,nan
